# Capítulo 8: Reducción de Dimensionalidades

## Reducción de Dimensionalidad — Qué es y para qué sirve

---

## 1. PCA (Análisis de Componentes Principales)
**Qué es:**  
Un método de reducción de dimensionalidad que proyecta los datos en nuevas variables (componentes principales) que capturan la mayor parte de la variación del dataset.

**Para qué sirve:**  
- Reducir el número de características manteniendo la mayor información posible.  
- Visualizar datos de alta dimensión.  
- Acelerar modelos de machine learning al eliminar redundancia.  
- Mitigar el ruido en los datos.

---

### 1.1 PCA con Scikit-Learn
**Qué es:**  
La implementación de PCA en la librería scikit-learn, que calcula automáticamente los componentes principales y las proyecciones.

**Para qué sirve:**  
- Aplicar PCA de forma sencilla y eficiente.  
- Integrarlo con pipelines y otros modelos de ML.  
- Explorar cuánta varianza se retiene según la cantidad de componentes.

---

### 1.2 PCA Incremental
**Qué es:**  
Una variante del PCA estándar que procesa los datos por lotes, sin necesidad de cargar todo el dataset en memoria.

**Para qué sirve:**  
- Trabajar con datasets muy grandes o que no caben en RAM.  
- Realizar reducción de dimensionalidad en flujos continuos de datos.

---

## 2. Kernel PCA
**Qué es:**  
Una extensión no lineal de PCA que utiliza funciones kernel para proyectar los datos en un espacio de mayor dimensión donde patrones complejos pueden volverse lineales.

**Para qué sirve:**  
- Capturar estructuras no lineales que el PCA tradicional no puede.  
- Mejorar tareas de clasificación o clustering tras la reducción.  
- Visualizar datos con forma de espiral, curvas o patrones complejos.

---

### 2.1 Seleccionar un Kernel
**Qué es:**  
El proceso de elegir una función kernel (RBF, sigmoide, polinomial, etc.) para definir cómo se miden las similitudes en Kernel PCA.

**Para qué sirve:**  
- Controlar el tipo de transformación no lineal aplicada.  
- Ajustar el modelo para detectar estructuras específicas en los datos.  
- Mejorar la separación entre grupos o clases en el espacio reducido.

---

## 3. LLE (Locally Linear Embedding)
**Qué es:**  
Un método de reducción de dimensionalidad no lineal que preserva las relaciones locales entre los puntos. En lugar de buscar componentes globales como PCA, se enfoca en reconstruir cada punto a partir de sus vecinos cercanos.

**Para qué sirve:**  
- Visualizar datos que se encuentran en variedades (manifolds) no lineales.  
- Capturar estructuras complejas preservando la geometría local.  
- Es especialmente útil cuando la forma de los datos es curva o intrincada (ej., "Swiss roll").



## <span style="color:green">1. PCA</span>

PCA=Principal Component Analysis

In [1]:
#Importa los librerías pandas, numpy, matplotlib, os.

In [2]:
#Trae los datos que necesitamos
#candidates = {'gmat': [780,750,690,710,680,730,690,720,740,690,610,690,710,680,770,610,580,650,540,590,620,600,550,550,570,670,660,580,650,660,640,620,660,660,680,650,670,580,590,690],
             # 'gpa': [4,3.9,3.3,3.7,3.9,3.7,2.3,3.3,3.3,1.7,2.7,3.7,3.7,3.3,3.3,3,2.7,3.7,2.7,2.3,3.3,2,2.3,2.7,3,3.3,3.7,2.3,3.7,3.3,3,2.7,4,3.3,3.3,2.3,2.7,3.3,1.7,3.7],
             # 'work_experience': [3,4,3,5,4,6,1,4,5,1,3,5,6,4,3,1,4,6,2,3,2,1,4,1,2,6,4,2,6,5,1,2,4,6,5,1,2,1,4,5],
             # 'admitted': [1,1,0,1,0,1,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,1,1,0,0,1,1,1,0,0,0,0,1]
             # }


In [3]:
#Separa el dataframe en los datos que vamos a utilizar para predecir y los datos predichos

In [4]:
#Una vez teniendo nuestros datos centramos x restándole su media.

#Recordemos que SVD descompone X en 3 matrices U, E y V por lo que utilizamos la función de numpy svd.

#V es la que contiene los vectores con los componentes principales
#para obtener los primeros 2 simplemente transponemos sus primeras 2 columnas.


Para proyectar nuestros datos en el hiperplano y obtener nuestra matriz X de dimensiones reducidas
tenemos que XNUEVA = XW donde X es nuestra matriz de datos original y W es la matriz 
que contiene nuestros vectores de componentes principales.

In [5]:
#

### <span style="color:blue">1.1 Con Scikit</span>

In [6]:
#Importa PCA

#Genera el objeto 


In [9]:
#Calcula la distribución de la varianza  


Esta distribución nos indica la proporción de varianza que cada componente principal contiene en comparación al set de datos original.

In [8]:
# Crea  un modelo donde tengas al menos el 90% de la varianza consevada


***

In [10]:
#Descomprime los datos con la función inverse_transform

#Despliega el resultado de la descompresión


Se pierde fidelidad porque no explica la variación en todos los datos pero es suficiente para entrenar al algoritmo

***

### <span style="color:blue">1.2 PCA Incremental</span>

In [11]:
#Importa IncrementalPCA

#Divide los datos en 3, es decir, 3 subsets. 


***

## <span style="color:green">2. Kernel PCA</span>

In [12]:
#Importa KernelPCA

#Haz un rbf= Radial Basis function 


In [13]:

#Visualiza los resultado 


### <span style="color:blue">2.1 Seleccionar un Kernel</span>

In [14]:
#Importa GridSearchCV

#Importa LogisticRegressin

#Importa Pipeline


#Arma un pipeline que pase por KernelPCA y LogisticRegression


In [15]:
#Corre el GridSearch


In [16]:
#Ejecuta el GridSearch


In [17]:
#Imprime el mejor parámetro que equivale al PCA que será la mejor regresión logística


## <span style="color:green">2. LLE</span>

LLE= Locally Linear Embedding

Es un método de reducción de dimensionalidad no lineal y **no depende de proyecciones**

Funciona midiendo como cada instancia de entrenamiento se relaciona linealmente con sus instancias vecinas y busca una representación lineal de pocas dimensiones del set de datos donde estas relaciones entre instancias cercanas o vecinas están bien preservadas. 

In [19]:
#Importa LocallyLinearEmbedding

#Selecciona el número de dimensiones, componentes y "vecinos" 
